<a href="https://colab.research.google.com/github/gulliyevn/ML-NLP-LLMProjects/blob/main/TraderPlusOnline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import requests

def fetch_twelvedata(symbol="XBR/USD", interval="1h", apikey="2f88eb1e7f9b49ef884557f27c95bd37", outputsize=5000):
    """
    Получает исторические данные с TwelveData по указанному символу.
    """
    url = f"https://api.twelvedata.com/time_series"
    params = {
        "symbol": symbol,
        "interval": interval,
        "apikey": apikey,
        "outputsize": outputsize,
        "format": "JSON"
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "values" not in data:
        raise ValueError(f"Ошибка API: {data.get('message', 'No data returned')}")

    df = pd.DataFrame(data["values"])
    df["datetime"] = pd.to_datetime(df["datetime"])
    df = df.sort_values("datetime")
    df = df.set_index("datetime")

    # Преобразование в float
    df = df.astype(float)

    print(f"✅ Загрузка завершена: {symbol}, {interval}, {len(df)} строк")
    return df

# Пример: загружаем нефть Brent (XBR/USD)
brent_df = fetch_twelvedata("XBR/USD", interval="1h")
brent_df.head()

✅ Загрузка завершена: XBR/USD, 1h, 5000 строк


,open,high,low,close
datetime,,,,
2024-08-12 19:00:00,79.870000,80.000000,79.610000,79.99
2024-08-12 20:00:00,80.000000,80.040001,79.880000,79.97
2024-08-12 21:00:00,79.970000,80.089996,79.940000,80.07
2024-08-12 22:00:00,80.059998,80.240000,79.930000,80.24
2024-08-12 23:00:00,80.220000,80.440000,80.029999,80.17


In [ ]:
brent_df.to_csv("brent_1h_raw.csv")

In [ ]:
import numpy as np
import pandas as pd

def add_technical_indicators(df):
    """
    Добавляет RSI, MACD, BB, ADX, ATR и другие индикаторы в датафрейм.
    """
    df = df.copy()

    # 1. RSI (14)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / (loss + 1e-9)
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # 2. MACD (12-26)
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # 3. Bollinger Bands (20)
    sma20 = df['close'].rolling(window=20).mean()
    std20 = df['close'].rolling(window=20).std()
    df['BB_upper'] = sma20 + 2 * std20
    df['BB_lower'] = sma20 - 2 * std20

    # 4. ATR (14)
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = tr.rolling(window=14).mean()

    # 5. ADX (14)
    plus_dm = df['high'].diff()
    minus_dm = df['low'].diff().abs()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm < 0] = 0
    tr14 = tr.rolling(window=14).sum()
    plus_di14 = 100 * (plus_dm.rolling(window=14).sum() / tr14)
    minus_di14 = 100 * (minus_dm.rolling(window=14).sum() / tr14)
    dx = 100 * (np.abs(plus_di14 - minus_di14) / (plus_di14 + minus_di14 + 1e-9))
    df['ADX_14'] = dx.rolling(window=14).mean()

    # 6. SMA / EMA (тренды)
    df['SMA_20'] = df['close'].rolling(window=20).mean()
    df['EMA_50'] = df['close'].ewm(span=50, adjust=False).mean()

    print("📈 Индикаторы успешно добавлены")
    return df

In [ ]:
brent_indicators = add_technical_indicators(brent_df)
brent_indicators.tail(3)

📈 Индикаторы успешно добавлены


,open,high,low,close,RSI_14,MACD,MACD_signal,BB_upper,BB_lower,ATR_14,ADX_14,SMA_20,EMA_50
datetime,,,,,,,,,,,,,
2025-06-18 04:00:00,74.93,76.890000,74.820000,76.73,67.300718,0.626759,0.396540,76.297609,71.837391,1.727857,18.991014,74.0675,73.526101
2025-06-18 05:00:00,76.74,76.890000,75.099998,76.46,63.430721,0.714633,0.460159,76.631003,71.694997,1.764286,19.301495,74.1630,73.641156
2025-06-18 06:00:00,76.47,77.080002,75.170000,76.89,69.717616,0.809638,0.530055,77.024704,71.717296,1.796429,18.743761,74.3710,73.768561


In [ ]:
def add_smc_features(df):
    """
    Добавляет простые SMC-признаки: OB, FVG, CHoCH, BOS
    """
    df = df.copy()

    # 1. Order Blocks (OB)
    df['OB_Bullish'] = ((df['close'].shift(1) < df['open'].shift(1)) &
                        (df['close'] > df['open'])).astype(int)

    df['OB_Bearish'] = ((df['close'].shift(1) > df['open'].shift(1)) &
                        (df['close'] < df['open'])).astype(int)

    # 2. FVG: Разрыв между low текущей свечи и high предыдущей
    df['FVG_Up'] = (df['low'] > df['high'].shift(1)).astype(int)
    df['FVG_Down'] = (df['high'] < df['low'].shift(1)).astype(int)

    # 3. CHoCH — смена структуры (пересечение локального max/min)
    df['CHoCH_Bullish'] = (df['close'] > df['high'].shift(1)).astype(int)
    df['CHoCH_Bearish'] = (df['close'] < df['low'].shift(1)).astype(int)

    # 4. BOS (break of structure) — price breaks previous swing high/low
    df['BOS_High'] = (df['high'] > df['high'].shift(1).rolling(5).max()).astype(int)
    df['BOS_Low'] = (df['low'] < df['low'].shift(1).rolling(5).min()).astype(int)

    print("🔍 SMC признаки успешно добавлены")
    return df

In [ ]:
brent_smc = add_smc_features(brent_indicators)
brent_smc.tail(3)

🔍 SMC признаки успешно добавлены


,open,high,low,close,RSI_14,MACD,MACD_signal,BB_upper,BB_lower,ATR_14,...,SMA_20,EMA_50,OB_Bullish,OB_Bearish,FVG_Up,FVG_Down,CHoCH_Bullish,CHoCH_Bearish,BOS_High,BOS_Low
datetime,,,,,,,,,,,,,,,,,,,,,
2025-06-18 04:00:00,74.93,76.890000,74.820000,76.73,67.300718,0.626759,0.396540,76.297609,71.837391,1.727857,...,74.0675,73.526101,0,0,0,0,1,0,1,0
2025-06-18 05:00:00,76.74,76.890000,75.099998,76.46,63.430721,0.714633,0.460159,76.631003,71.694997,1.764286,...,74.1630,73.641156,0,1,0,0,0,0,0,0
2025-06-18 06:00:00,76.47,77.080002,75.170000,76.89,69.717616,0.809638,0.530055,77.024704,71.717296,1.796429,...,74.3710,73.768561,1,0,0,0,0,0,1,0


In [ ]:
def create_target_labels(df, threshold=0.0015, future_shift=4):
    """
    Создает таргет на основе изменения цены через future_shift шагов
    """
    df = df.copy()
    df['future_close'] = df['close'].shift(-future_shift)
    df['price_change'] = (df['future_close'] - df['close']) / df['close']

    df['Target'] = 1  # по умолчанию - Sideways
    df.loc[df['price_change'] > threshold, 'Target'] = 2  # Up
    df.loc[df['price_change'] < -threshold, 'Target'] = 0  # Down

    df.drop(['future_close', 'price_change'], axis=1, inplace=True)

    print("🎯 Таргет успешно создан")
    return df

In [ ]:
brent_labeled = create_target_labels(brent_smc)
brent_labeled[['close', 'Target']].tail(10)

🎯 Таргет успешно создан


,close,Target
datetime,,
2025-06-17 21:00:00,74.800000,0
2025-06-17 22:00:00,73.580000,2
2025-06-17 23:00:00,75.280000,0
2025-06-18 00:00:00,75.080002,2
2025-06-18 01:00:00,74.099998,2
2025-06-18 02:00:00,75.360000,2
2025-06-18 03:00:00,74.920000,1
2025-06-18 04:00:00,76.730000,1
2025-06-18 05:00:00,76.460000,1


In [ ]:
from sklearn.preprocessing import RobustScaler

def prepare_lstm_sequences(data, feature_cols, sequence_length=96, target_col='Target'):
    """
    Подготовка последовательностей для LSTM модели
    """
    # Масштабирование
    scaler = RobustScaler()
    scaled_features = scaler.fit_transform(data[feature_cols])

    X, y = [], []

    for i in range(sequence_length, len(scaled_features)):
        X.append(scaled_features[i-sequence_length:i])
        y.append(data[target_col].iloc[i])

    X = np.array(X)
    y = np.array(y)

    print(f"✅ Последовательности созданы:")
    print(f"   X shape: {X.shape}")
    print(f"   y shape: {y.shape}")

    X_lstm, y_lstm, scaler = prepare_lstm_sequences(
    data=brent_labeled,
    feature_cols=feature_columns,
    sequence_length=96,
    target_col='Target')

    return X, y, scaler

# ✅ Выбор фичей
feature_columns = [
    'open', 'high', 'low', 'close',
    'RSI_14', 'MACD', 'MACD_signal',
    'BB_upper', 'BB_lower', 'ATR_14', 'ADX_14',
    'SMA_20', 'EMA_50',
    'OB_Bullish', 'OB_Bearish',
    'CHoCH_Bullish', 'CHoCH_Bearish'
]


In [ ]:
print("NaN в X_lstm:", np.isnan(X_lstm).sum())

In [ ]:
print("X_lstm mean:", np.mean(X_lstm))
print("X_lstm std:", np.std(X_lstm))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(LSTM(64))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("✅ LSTM-модель создана")
    return model

In [ ]:
# Train-test split
split_idx = int(len(X_lstm) * 0.8)
X_train_lstm = X_lstm[:split_idx]
X_test_lstm = X_lstm[split_idx:]
y_train_lstm = y_lstm[:split_idx]
y_test_lstm = y_lstm[split_idx:]

# Создание модели
model = create_lstm_model(input_shape=X_lstm.shape[1:], num_classes=3)

# Обучение
history = model.fit(
    X_train_lstm, tf.keras.utils.to_categorical(y_train_lstm, 3),
    validation_data=(X_test_lstm, tf.keras.utils.to_categorical(y_test_lstm, 3)),
    epochs=30, batch_size=32, verbose=1
)